<h1>RadxRate Tutorial</h1>

---

This interactive tutorial takes you through the steps of how to run RadxRate. RadxRate builds upon the capabilities of RadxKdp and RadxPid by calculating hourly rain rates at each radar gate depending on the local PID category and polarimetric values. It shares parameters files with those two applications. RadxRate will calculate the rain rates with either raw or attenuated-corrected data; the user can choose the option they prefer. The three-dimensional rain rates produced by RadxRate can be then used in RadxQpe to estimate the rain rate closest to the surface, after accounting for beam blockage, noise, and clutter.

---

*Note: this tutorial is just one type of workflow to use RadxRate. There are several other workflows!*

<h2>Overview</h2>


**1. Parameter files**
* RadxRate main params
* RadxRate Kdp_specific params
* RadxRate Pid_specific params
* RadxRate Rate_specific params
* Pid Thresholds params (Radar d
ependent)
* Grib2toMdv params

### **Parameter files modifications**
* **RadxRate main params**
    * input_dir (radar data) [line 105]
    * INPUT FIELD INFORMATION. [line 155]
        * SNR_available = TRUE; change to FALSE; [line 177]
        * DBZ_field_name = "DBZ"; change to "REF"; [line 215]
        * LDR_available = TRUE; change to FALSE [line 255]
    * kdp_params_file_path  [line 284]
    * pid_params_file_path  [line 303]
    * PID_use_attenuation_corrected_fields = TRUE [line 331]
    * RATE_params_file_path [line 350]
    * RATE_use_attenuation_corrected_fields = TRUE [line 378]
    * output_dir [line 699] -- or set in command line with -outdir flag
    * output_format = OUTPUT_FORMAT_CFRADIAL; (Default) [line 755] 
* **RadxRate Kdp_specific params**
    * KDP_fir_filter_len = KDP_FIR_LEN_10; [line 65]
    * KDP_psob_method = PEAK_REMOVAL_METHOD; [line 114]
* **RadxRate Pid_specific params**
    * PID_thresholds_file_path [line 27]
    * PID_use_soundings_from_spdb = FALSE; set to TRUE[line 303]
    * PID_sounding_spdb_url = "spdb/soundings"; [line 313]
    * PID_sounding_location_name [line 339]
    *  **do sounding steps first before filling Pid variables out**
* **RadxRate Rate_specific params**
    * N/A?
* **Pid Thresholds params**
    * Radar dependent
        * Nexrad is S-band simultaneous h/v transmission and recieving


https://stackoverflow.com/questions/19699059/print-directory-file-structure-with-icons-for-representation-in-markdown
```
BASE_DIR
│   RadxRate_tutorial.ipynb
│ 
│
└───data
│   │   ...
│   │   
│   │
│   │───raw
│   │   │   KMHX20180914_191822_V06.ar2v
│   │   │   gfsanl_4_2018091418.g2.tar
│   │   │   
│   │───CfRadial
│   │   │   ...
│   │   │
│   │   │
│   │───sounding 
│   │   │   ...
│   │   │
│    ───
│   
└───params
│   │   Pid_thresholds.sband.shv.txt
│   │   RadxRate_main_params
│   │   RadxRate_Kdp_params
│   │   RadxRate_Pid_params
│   │   RadxRate_rate_params



- Print out parameter files

In [ ]:
# # create original parameter files
# !/usr/local/lrose/bin/RadxRate -print_params > ./params_ori/RadxRate_main_params
# !/usr/local/lrose/bin/RadxRate -print_params_kdp > ./params_ori/RadxRate_Kdp_params
# !/usr/local/lrose/bin/RadxRate -print_params_pid > ./params_ori/RadxRate_Pid_params
# !/usr/local/lrose/bin/RadxRate -print_params_rate > ./params_ori/RadxRate_rate_params

# # setup parameter files for Users
# !/usr/local/lrose/bin/RadxRate -print_params > ./params/RadxRate_main_params
# !/usr/local/lrose/bin/RadxRate -print_params_kdp > ./params/RadxRate_Kdp_params
# !/usr/local/lrose/bin/RadxRate -print_params_pid > ./params/RadxRate_Pid_params
# !/usr/local/lrose/bin/RadxRate -print_params_rate > ./params/RadxRate_rate_params
# !/usr/local/lrose/bin/Grib2toMdv -print_params > ./params/Grib2toMdv_params

In [ ]:
import os

Here we will set up the directory structure to simplify our commands. (*note: modify BASE_DIR and LROSE_DIR if your personal machine has different directory structures*)

* BASE_DIR: the base directory containing the directories for the notebooks, data, parameter files
* RADAR_NAME: the name of the radar used in this tutorial
* LROSE_DIR: path to the LROSE installation 

In [ ]:
os.environ['BASE_DIR'] = '/home/jovyan/ams2023'
os.environ['RADAR_NAME'] = 'KMHX'
os.environ['LROSE_DIR'] = '/usr/local/lrose/bin'
!echo "Base directory: "$BASE_DIR
!echo "Base directory: "$RADAR_NAME

# Data download and directory organization

https://drive.google.com/drive/folders/1nQ1-CUR_OFuSdWgsE-qL35L5rcWTmEh4?usp=share_link

In [ ]:
### make a directory for all the data files (raw and analysis)
# !rm -rf ${BASE_DIR}/data/echo
# !mkdir ${BASE_DIR}/data/echo

### make subdirectory within data for the raw data
# !rm -rf ${BASE_DIR}/data/echo/raw
# !mkdir ${BASE_DIR}/data/echo/raw

### make subdirectory within data for sounding data
# !rm -rf ${BASE_DIR}/data/echo/sounding
# !mkdir ${BASE_DIR}/data/echo/sounding

### wget raw files from ftpserver and plop in appropriate $BASE_DIR subdirectory
# !cd ${BASE_DIR}/data/echo/raw; wget ftpserver/KMHX20180914_191822_V06.ar2v
# !cd ${BASE_DIR}/data/echo/raw; wget ftpserver/gfsanl_4_2018091418.g2.tar

# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1NT_gA3RPi5PB0PSCh8b8UNm43uc5fQrO' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1NT_gA3RPi5PB0PSCh8b8UNm43uc5fQrO" -O gfsanl_4_2018091418.g2.tar && rm -rf /tmp/cookies.txt
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=10On5qmT4p-Wvm1kpdGjLUHdu0TQyXZDK' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=10On5qmT4p-Wvm1kpdGjLUHdu0TQyXZDK" -O KMHX20180914_191822_V06.ar2v && rm -rf /tmp/cookies.txt

### move files to proper directories
# !mv KMHX20180914_191822_V06.ar2v ${BASE_DIR}/data/echo/raw
# !mv gfsanl_4_2018091418.g2.tar ${BASE_DIR}/data/echo/raw

# Convert Nexrad Level II to CfRadial files

In [ ]:
# !rm -rf ${BASE_DIR}/data/echo/CfRadial
# !mkdir ${BASE_DIR}/data/echo/CfRadial
!${LROSE_DIR}/RadxConvert -f ${BASE_DIR}/data/echo/raw/KMHX20180914_191822_V06.ar2v -outdir ${BASE_DIR}/data/echo/CfRadial

# Untar GFS analysis grib2 files for sounding
* Grb2 --> Mdv --> Spdb

In [ ]:
# Grib2Mdv --> Mdv2SoundingSpdb

# make directory for grib files in data/sounding
!rm -rf ${BASE_DIR}/data/echo/sounding/grib
!mkdir ${BASE_DIR}/data/echo/sounding/grib

# extract file from tape archive (tar)
!tar -xvf ${BASE_DIR}/data/echo/raw/gfsanl_4_2018091418.g2.tar -C ${BASE_DIR}/data/sounding/grib/

- **Grib2toMdv**
    - input_dir [line 69] optional, if directory is specified in command line
    - write_non_forecast [line 385]
    - non_forecast_mdv_url (location of outputted mdv files) [line 395]
    - out_projection_info [line 715] Radar location 
    - out_grid_info [line 749] Subjective box around radar(s)
    - height_levels [line 838] Height levels, optional

In [ ]:
# convert model analysis to Mdv
!${LROSE_DIR}/Grib2toMdv -params ${BASE_DIR}/params/echo/Grib2toMdv_params -f ${BASE_DIR}/data/echo/sounding/grib/*.grb2

- **Mdv2SoundingSpdb**
    - input data field names [line 126] 
    - sampling locations [line 264]
        - Need to input location of current radar KMHX
    - ouput_url [line 363]

In [ ]:
# convert Mdv to SoundingSpdb
!${LROSE_DIR}/Mdv2SoundingSpdb -params ${BASE_DIR}/params/echo/Mdv2SoundingSpdb_params -f ${BASE_DIR}/data/echo/sounding/mdv/20180914/*.mdv*
!${LROSE_DIR}/Mdv2SoundingSpdb -params ${BASE_DIR}/params/echo/Mdv2SoundingSpdb_params -f ${BASE_DIR}/data/echo/sounding/mdv/20180915/*.mdv*

- Check sounding data is filled (except vertical velocity and divergence)

In [ ]:
!${LROSE_DIR}/SpdbQuery -url ${BASE_DIR}/data/echo/sounding/spdb -start "2018 09 14 00 00 00" -end " 2018 09 14 23 00 00"

# Setup steps complete, run RadxRate

In [ ]:
!${LROSE_DIR}/RadxRate -params ${BASE_DIR}/params/echo/RadxRate_main_params -f ${BASE_DIR}/data/echo/CfRadial/20180914/*.nc -outdir ${BASE_DIR}/data/echo/rate

